<a href="https://colab.research.google.com/github/FarukEmreCiger/ImageColorization/blob/master/ImageColorizationTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile

Using TensorFlow backend.


In [0]:
torch.cuda.get_device_name(0)

NameError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
project_path='/content/drive/My Drive/NeuralNetworkProject/'
train_path = project_path+'Train/'
npy_files_path=project_path+'NpyFiles/'
check_point_path=project_path+'Weights/'
test_path = project_path+'TestSmall/'
result_path = project_path+'result/'

In [0]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train = train_datagen.flow_from_directory(train_path, target_size=(256, 256),batch_size=6020,class_mode=None)

In [0]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)
np.save(npy_files_path + 'dataX.npy', X)
np.save(npy_files_path + 'dataY.npy', Y)

In [0]:
X=np.load(npy_files_path + 'dataX.npy')
Y=np.load(npy_files_path + 'dataY.npy')

In [0]:
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)

In [0]:
#ADDED
from keras.callbacks import *
from datetime import datetime
#'epochs:{epoch:03d}-val_accuracy:{val_accuracy:.3f}.hdf5'
checkpoint = ModelCheckpoint(check_point_path + 'bestWeights2.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
logdir = check_point_path +  "Log"+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
callbacks_list = [checkpoint,tensorboard_callback]

In [0]:
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])


In [0]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, load_model
model=load_model(check_point_path + 'bestWeights.hdf5')

In [0]:
#model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.fit(X,Y,validation_split=0.2, epochs=80,callbacks=callbacks_list)#ADDED CALLBACK

Train on 4816 samples, validate on 1204 samples
Epoch 1/80
 128/4816 [..............................] - ETA: 1:22:54 - loss: 0.0263 - accuracy: 0.6551

KeyboardInterrupt: ignored

In [0]:
test = os.listdir(test_path)
color_me = []
for imgName in test:
  img = img_to_array(load_img(test_path + imgName))
  img = resize(img ,(256,256))
  color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
print(color_me.shape)

In [0]:
output = model.predict(color_me)
output = output * 128

In [0]:
# Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    imsave(result_path + str(i) + ".png" , lab2rgb(result))

In [0]:
while(1)
  acikTut=1